In [2]:
import keras
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.regularizers import *
import os

In [3]:
main_dataset_dir = "brain_tumor_dataset/brain_tumor_classification/"
os.listdir(main_dataset_dir + "Training")


['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

In [4]:
main_dataset_labels = np.array(['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor'])

In [5]:
train_dataset = image_dataset_from_directory(
    directory=main_dataset_dir+"Training",
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=74,
    validation_split=0.2,
    subset="training"
)

# Normalize the images using tf.map()
train_dataset = train_dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))

validation_dataset = image_dataset_from_directory(
    directory=main_dataset_dir+"Training",
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=74,
    validation_split=0.2,
    subset="validation"
)

# Normalize the images using tf.map()
validation_dataset = validation_dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))

test_dataset = image_dataset_from_directory(
    directory=main_dataset_dir+"Testing",
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=74,   
)

# Normalize the images using tf.map()
test_dataset = test_dataset.map(lambda x, y: (tf.cast(x, tf.float32)/255.0,y))

Found 2870 files belonging to 4 classes.
Using 2296 files for training.
Found 2870 files belonging to 4 classes.
Using 574 files for validation.
Found 394 files belonging to 4 classes.


In [6]:
def VGG16_based_model():
    base_model = VGG16(
        weights = "imagenet", 
        input_shape = (256, 256, 3),
        include_top = False, 
    )

    base_model.trainable = False

    input = Input((256, 256, 3))
    x = RandomRotation(factor=(-0.2, 0.2))(input)
    x = RandomZoom(height_factor=0.1, width_factor=0.1)(x)
    x = RandomContrast(factor= (0.8, 1.2))(x)
    x = base_model(input, training=False)
    x = Flatten()(x)
    x = BatchNormalization()(x)
    x = Dense(1024, activation="relu")(x)
    output = Dense(4, activation="linear")(x)
    
    return Model(inputs=input, outputs=output),base_model

In [7]:
model_1, base_models_1 = VGG16_based_model()

In [8]:
model_1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 8, 8, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32768)          │       131,072 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │    33,555,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         4,100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,405,316 (184.65 MB)

 Trainable params: 33,625,092 (128.27 MB)

 Non-trainable params: 14,780,224 (56.38 MB)

In [9]:
model_1.compile(loss = CategoricalCrossentropy(from_logits=True), optimizer=Adam(), metrics=["accuracy"])

In [10]:
checkpoint_file_path_1 = "./Checkpoints/model_1.keras"
callbacks_1 = [
    EarlyStopping(
        monitor = "val_loss", 
        min_delta = 1e-3, 
        patience = 3,
        start_from_epoch = 10,
        restore_best_weights = True, 
    ),
    ModelCheckpoint(
        filepath = checkpoint_file_path_1,
        monitor = "val_loss",
        save_best_only = True,
    )
]

: 

In [ ]:
hist_1 = model_1.fit(train_dataset, epochs=5, callbacks=callbacks_1, validation_data=validation_dataset)


Epoch 1/5
